# Resume Optimizer using AI

In [ ]:
# Install Libs
!pip install openai markdown weasyprint

**Import the Libraries**

In [4]:
from IPython.display import display, Markdown
import os
import openai
from markdown import markdown
from weasyprint import HTML

# Connect to OpenAI
openai.api_key = os.getenv("OPENAI_KEY")

### Import Resume and Job Description

In [5]:
# Open and Read the resume.md 
with open("resume/morrison_resume.md", "r", encoding="utf-8") as file:
    resume_string = file.read()

display(Markdown(resume_string))

# Rod Morrison

*   Email: [**rod.morrison@gmail.com**](mailto:rod.morrison@gmail.com) 
*   Phone: [**713-907-7102**](tel:7139077102)
*   [**LinkedIn**](https://www.linkedin.com/in/rmorrison/)
*   [**Github**](https://github.com/Mojoryson)


**Professional Summary**
Data-driven growth leader with expertise in AI, data analytics, and cloud platforms. I specialize in transforming data
ecosystems to drive business growth, with a focus on optimizing customer acquisition, retention, and product
adoption. With a proven track record of scaling early-stage companies and modernizing legacy systems, I build
agile, real-time infrastructures that unlock revenue opportunities. Adept at leading high-performing teams to solve
complex technical challenges, I excel at driving rapid growth and successfully positioning companies for major
milestones, such as funding rounds or market expansions.

**Skills**

- **Technical Expertise:** Linux, Python, Bash, AWS, GCP, Vertex AI, SQL, H20.ai, Confluent Kafka, ETL/ELT, SaaS, Advanced Analytics, Data Fabric Design, Snowflake, Data Engineering, ML/AI, CI/CD, DCX, CEM, VOC
- **Sales:** MEDDIC, Strategic Partnership Management, Stakeholder Communication, Customer Acquisition and Retention, GTM Strategy, Performance Metrics, KPI Analysis, Multi-year Presidents Club & Quota Achievement
- **Leadership:** Skilled at leading cross-functional teams, mentoring, and scaling high-performing teams to drive business growth, enhance brand development, optimize user experience, , collaborate on product development and growth

**Experience**

**Raken Data Group – Head of Data Solutions 2024 - Present**

- Responsible for RDG’s expansion into North America, focusing on customer acquisition, retention, and revenue.
- Manage strategic partnerships with software providers to drive market share and enhance North American presence.
- Oversee end-to-end project delivery, ensuring timely execution and client satisfaction.
- Deliver platform-agnostic, bespoke solutions that generate ROI and drive customer satisfaction.
- Leverage data analytics to identify growth opportunities and optimize business performance.
- Plan, develop, and deploy RDG’s proprietary ML models into SaaS product offerings.

**HungerRush – Sales Engineer 2023 – 2023**

- Developed the technical sales, marketing, and GTM execution plan for a new GenAI ordering system.
- Nurture and grow Enterprise Accounts by aligning customers’ critical business issues with tailored solutions.
- Increased enterprise account retention by 25 % through strategic solution sales strategies.
- Manage installation, onboarding, and operationalization for the North American Enterprise book of business.
- Install, configure, troubleshoot, and support client deployments on-premises and in the Azure Cloud.

**Highspot – Manager, Solutions Consulting 2022 – 2023**

- Developed go-to-market readiness, identified gaps in preparedness, and built plans to ensure the team was
    trained and equipped to perform successfully in support of annual goals.
- Championed solution-selling mentorship and coaching using the Corporate Visions and MEDDIC
    methodologies.
- Work with teams to identify, qualify, and prioritize coverage for business opportunities.
- Participate in opportunity review meetings to provide insight into how to secure the technical win.
- Reinvented the go-to-market platform demonstrations to incorporate storytelling and real-world use cases.
- Maintain customer satisfaction by working cross-functionally to resolve product problems affecting customer satisfaction.
- Build trust with customers and influential relationships in accounts and be the technical sponsor and driver for strategic business opportunities.

**Contentsquare – Head of Solutions Consulting (East & LATAM) 2017 – 2022**

- Promoted from Senior Sales Engineer to lead the Solutions Consulting team for North America East and LATAM, driving significant YOY revenue growth.
- Bootstrapped and architected rapid growth in the US Central territory, achieving $3M in ARR over 12 months.
- Built and led a top-performing post-sales engineering team to drive adoption, customer retention, NPS, and CSAT scores, resulting in substantial revenue growth and expansion.
- Accelerated Latin America expansion, driving growth from $20K to $1M ARR in less than a year.
- Provide technical leadership to drive services and solution integration across multiple geographies in collaboration with Account Executives and Managers.
- Orchestrated technical sales campaigns that exceeded targets and boosted revenue and brand recognition.
- Developed and delivered comprehensive onboarding programs for new team members, mentoring, and coaching new hires on sales engineering skills.

**Datameer 2015 – 2017
Manager, Solutions Engineering**

- Developed and executed end-to-end sales and expansion strategies to drive new enterprise client acquisition while expanding existing customer relationships.
- Spearheaded Solutions Engineering team talent acquisition, onboarding, and skills development efforts.
- Developed and executed the GTM strategy for Latin America.
- Coordinated and structured the Sales Engineering Team’s account plans and business development efforts.
- Championed Customer Success and client relationship management to review product delivery and resolved implementation and adoption issues.
- Developed demonstrations, presentations, and other assets to support sales and marketing efforts.
- Created and hosted client-facing product awareness, education, and training programs.

**User Replay – Director of Sales Engineering 2015 – 2015**

- Led sales and account penetration as the 21st employee hired in an early-stage UK company.
- Achieved rapid growth, increasing North American territory revenue from $0 to $ 1 million ARR in 8 months.
- Aligned customer goals, needs, and growth plans with customized solutions to drive increased ACV.
- Served as the technical SME, providing critical support to the North American customer base.
- Identified and addressed customers' critical business issues to expand ACV and market reach.
- Developed and delivered compelling sales presentations and customized demonstrations to increase customer engagement.
- Guided customers to success by effectively facilitating proof of concepts and production deployments.

**Education & Certifications**

**Certifications**:
- [Google Cloud Public Profile](https://partner.cloudskillsboost.google/public_profiles/4e087ef0-ed5c-48b5-b763-8e5ade38459e)
- [Snowflake Sales & Technical](https://certificates.confluent.io/profile/shaded5458/wallet)
- [Confluent Kafka & Flink Sales & Technical](https://certificates.confluent.io/profile/shaded5458/wallet)
- Responsive Web Design, JavaScript
- Echelon Front Leadership Certification
- United States Army, 3rd Battalion / 11th Armored Cavalry Regiment | Bad Hersfeld, Germany
- Diploma in Computer Programming, Computer Learning Center, San Francisco


In [7]:
# input job description
jd_string = input()

In [10]:
# Prompt user for job data
jd_string = "" 
while not jd_string:  # Loop until we get some input
    print("To help me analyze this job description for you, please paste the full text below:")
    jd_string = input()
    if not jd_string:
        print("Oops! Looks like you didn't paste anything. Please try again.")
        
print(jd_string)

To help me analyze this job description for you, please paste the full text below:
About the job BforeAI is an innovative and rapidly expanding scale-up dedicated to deterring cybercrime through cutting-edge predictive and pre-emptive technologies. We harness the power of prescriptive AI to revolutionize the way we tackle cyber threats, particularly in the realm of brand protection.  Named by Gartner in 26 reports over the last 2 years, BforeAI is the industry's fastest, most accurate solution for automated protection against online fraud.  We are like weather forecasts for cyber threats. Join us in the fight for a safer cyberspace!  ✨ What's cool about this job  As a Solution Engineer, you'll be at the forefront of shaping how our cutting-edge technology is implemented and leveraged by prospects and customers. You'll have a pivotal, outward-facing role, representing BforeAI with professionalism and technical expertise as you help clients unlock the full potential of our predictive sol

Build the Prompt Template

In [11]:
prompt_template = lambda resume_string, jd_string : f"""
You are a professional resume optimization expert specializing in tailoring resumes to specific job descriptions. Your goal is to optimize my resume and provide actionable suggestions for improvement to align with the target role.

### Guidelines:
1. **Relevance**:  
   - Prioritize experiences, skills, and achievements **most relevant to the job description**.  
   - Remove or de-emphasize irrelevant details to ensure a **concise** and **targeted** resume.
   - Limit work experience section to 4-5 most relevant roles
   - Limit bullet points under each role to 3-4 most relevant impacts

2. **Action-Driven Results**:  
   - Use **strong action verbs** and **quantifiable results** (e.g., percentages, revenue, efficiency improvements) to highlight impact.  

3. **Keyword Optimization**:  
   - Integrate **keywords** and phrases from the job description naturally to optimize for ATS (Applicant Tracking Systems).  

4. **Additional Suggestions** *(If Gaps Exist)*:  
   - If the resume does not fully align with the job description, suggest:  
     1. **Additional technical or soft skills** that I could add to make my profile stronger.  
     2. **Certifications or courses** I could pursue to bridge the gap.  
     3. **Project ideas or experiences** that would better align with the role.  

5. **Formatting**:  
   - Output the tailored resume in **clean Markdown format**.  
   - Include an **"Additional Suggestions"** section at the end with actionable improvement recommendations.  
   - Aim for a resume length of around 750-1000 words.
   - Tailor the tone and style to match the target company and industry.

---

### Input:
- **My resume**:  
{resume_string}

- **The job description**:  
{jd_string}

---

### Output:  
 **Tailored Resume**:  
   - A resume in **Markdown format** that emphasizes relevant experience, skills, and achievements.  
   - Incorporates job description **keywords** to optimize for ATS.  
   - Uses strong language and is no longer than **two pages**.

 **Contact Information:**  <-- Separate section for contact info
    - Include all provided contact details (email, phone, LinkedIn, GitHub).

 **Certifications:**  <-- Separate section for certifications at the bottom
    - Include all provided certifications with links.

 **Additional Suggestions** *(if applicable)*:
   - Determine if the resume and experience is a **FIT** and a good candidate for this specific role.
   - List **skills** that could strengthen alignment with the role.  
   - Recommend **certifications or courses** to pursue.  
   - Suggest **specific projects or experiences** to develop.

 **Evaluate as if you are the hiring manager the resume and experience against the job description to determine if the candidate is:
   - **FIT**: Fully meets or exceeds requirements.
   - **CLOSE**: Meets most requirements with minor gaps.
   - **STRETCH**: Has significant gaps but potential to grow into the role.

"""

In [12]:
# List all available models
models = openai.Model.list()

# Print the names of the models
for model in models['data']:
    print(model['id'])

gpt-4o-audio-preview-2024-10-01
gpt-4o-realtime-preview
gpt-4o-realtime-preview-2024-10-01
o1-mini-2024-09-12
dall-e-2
gpt-4-turbo
gpt-4-1106-preview
gpt-3.5-turbo
gpt-3.5-turbo-0125
gpt-3.5-turbo-instruct
babbage-002
davinci-002
whisper-1
dall-e-3
text-embedding-3-small
gpt-3.5-turbo-16k
gpt-4-0125-preview
gpt-4-turbo-preview
omni-moderation-latest
gpt-4o-2024-05-13
omni-moderation-2024-09-26
tts-1-hd-1106
chatgpt-4o-latest
gpt-4
gpt-4-0613
o1-mini
o1-preview
o1-preview-2024-09-12
tts-1-hd
text-embedding-ada-002
gpt-3.5-turbo-1106
gpt-4o-audio-preview
tts-1
tts-1-1106
gpt-3.5-turbo-instruct-0914
text-embedding-3-large
gpt-4o-realtime-preview-2024-12-17
gpt-4o-mini-realtime-preview
gpt-4o-mini-realtime-preview-2024-12-17
gpt-4o-2024-11-20
gpt-4o-mini
gpt-4o-audio-preview-2024-12-17
gpt-4o-mini-2024-07-18
gpt-4o-mini-audio-preview
gpt-4o-2024-08-06
gpt-4o-mini-audio-preview-2024-12-17
gpt-4o
gpt-4-turbo-2024-04-09


In [13]:
prompt = prompt_template(resume_string, jd_string)

In [17]:
# Make API call
response = openai.ChatCompletion.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "Expert resume writer"},
        {"role": "user", "content": prompt}
    ],
    temperature=0.7
)

# Extract the response content
generated_text = response.choices[0].message.content # FIX VARIABLE NAME

In [18]:
print(generated_text) # fix this

```markdown
# Rod Morrison

* Email: [**rod.morrison@gmail.com**](mailto:rod.morrison@gmail.com)  
* Phone: [**713-907-7102**](tel:7139077102)  
* [**LinkedIn**](https://www.linkedin.com/in/rmorrison/)  
* [**GitHub**](https://github.com/Mojoryson)  

---

## Professional Summary

Data-driven growth leader with extensive experience in AI, data analytics, and cloud platforms within cybersecurity contexts. Proven track record in transforming data ecosystems to optimize customer acquisition, retention, and product adoption. Expertise in driving the integration of predictive technologies, with an emphasis on enhancing security measures against cyber threats. Adept at leading cross-functional teams to tackle complex technical challenges and ensure seamless deployment of innovative solutions.

---

## Skills

- **Technical Expertise:** Python, AWS, GCP, SQL, ETL/ELT, API Integration, Data Analytics, Cloud Hosting Architectures, Cybersecurity Tools
- **Sales & Solutions Engineering:** Pre-sal

### Display the results

In [19]:
resume_list = generated_text.split("## Additional Suggestions")

In [20]:
display(Markdown(resume_list[0]))

```markdown
# Rod Morrison

* Email: [**rod.morrison@gmail.com**](mailto:rod.morrison@gmail.com)  
* Phone: [**713-907-7102**](tel:7139077102)  
* [**LinkedIn**](https://www.linkedin.com/in/rmorrison/)  
* [**GitHub**](https://github.com/Mojoryson)  

---

## Professional Summary

Data-driven growth leader with extensive experience in AI, data analytics, and cloud platforms within cybersecurity contexts. Proven track record in transforming data ecosystems to optimize customer acquisition, retention, and product adoption. Expertise in driving the integration of predictive technologies, with an emphasis on enhancing security measures against cyber threats. Adept at leading cross-functional teams to tackle complex technical challenges and ensure seamless deployment of innovative solutions.

---

## Skills

- **Technical Expertise:** Python, AWS, GCP, SQL, ETL/ELT, API Integration, Data Analytics, Cloud Hosting Architectures, Cybersecurity Tools
- **Sales & Solutions Engineering:** Pre-sales Engineering, Technical Consulting, Customer Engagement, Solution Design, Proof-of-Value Management, API Development
- **Leadership & Collaboration:** Cross-functional Team Leadership, Client Relationship Management, Training & Mentoring, Strategic Communication

---

## Experience

**Raken Data Group – Head of Data Solutions**  
*2024 - Present*  
- Spearheaded the expansion into North America, driving customer acquisition and enhancing revenue streams through tailored solutions.
- Managed strategic partnerships with software providers, significantly boosting market share and brand recognition in the cybersecurity sector.
- Leveraged data analytics to identify growth opportunities while ensuring that client systems are effectively integrated with RDG's predictive solutions.

**HungerRush – Sales Engineer**  
*2023*  
- Developed and executed a technical sales strategy for a GenAI ordering system, aligning customer technology stacks with innovative solutions.
- Increased enterprise account retention by 25% through effective solution-selling techniques and integration support within Azure Cloud environments.
- Facilitated seamless onboarding and operationalization for clients, enhancing their experience with the product.

**Highspot – Manager, Solutions Consulting**  
*2022 - 2023*  
- Led go-to-market readiness initiatives, ensuring the team was equipped to support annual growth goals through effective solution-selling methodologies.
- Designed impactful demonstrations that showcased the value of products, incorporating technical insights and customer stories to deepen engagement.
- Collaborated with cross-functional teams to resolve product-related issues, maintaining high levels of customer satisfaction.

**Contentsquare – Head of Solutions Consulting (East & LATAM)**  
*2017 - 2022*  
- Drove significant YOY revenue growth by architecting rapid expansion strategies across North America and Latin America, achieving $3M in ARR.
- Enhanced customer satisfaction and retention through the development of tailored onboarding programs and solution integration.
- Championed technical sales campaigns that consistently exceeded targets, reinforcing brand recognition in the market.

---

## Education & Certifications

- **Certifications**:  
  - [Google Cloud Public Profile](https://partner.cloudskillsboost.google/public_profiles/4e087ef0-ed5c-48b5-b763-8e5ade38459e)  
  - [Snowflake Sales & Technical](https://certificates.confluent.io/profile/shaded5458/wallet)  
  - [Confluent Kafka & Flink Sales & Technical](https://certificates.confluent.io/profile/shaded5458/wallet)  
  - Responsive Web Design, JavaScript  
  - Echelon Front Leadership Certification  

---



### Save File to PDF & HTML

In [21]:
# Save to PDF
output_resume_pdf = "resume/openai_resume_new.pdf"

# Convert Markdown to HTML
html_output = markdown(resume_list[0])
print(html_output)

<p>```markdown</p>
<h1>Rod Morrison</h1>
<ul>
<li>Email: <a href="mailto:rod.morrison@gmail.com"><strong>rod.morrison@gmail.com</strong></a>  </li>
<li>Phone: <a href="tel:7139077102"><strong>713-907-7102</strong></a>  </li>
<li><a href="https://www.linkedin.com/in/rmorrison/"><strong>LinkedIn</strong></a>  </li>
<li><a href="https://github.com/Mojoryson"><strong>GitHub</strong></a>  </li>
</ul>
<hr />
<h2>Professional Summary</h2>
<p>Data-driven growth leader with extensive experience in AI, data analytics, and cloud platforms within cybersecurity contexts. Proven track record in transforming data ecosystems to optimize customer acquisition, retention, and product adoption. Expertise in driving the integration of predictive technologies, with an emphasis on enhancing security measures against cyber threats. Adept at leading cross-functional teams to tackle complex technical challenges and ensure seamless deployment of innovative solutions.</p>
<hr />
<h2>Skills</h2>
<ul>
<li><strong>T

In [23]:
from markdown import markdown

def markdown_to_html_with_css(markdown_text, css_file="styles.css", google_font="Roboto"):
  """
  Converts markdown text to HTML, applies an external CSS file, and adds a Google Font.

  Args:
    markdown_text: The markdown text to convert.
    css_file: The path to the CSS file.
    google_font: The name of the Google Font to use.

  Returns:
    The HTML output as a string.
  """

  html_output = markdown(markdown_text)

  # Construct the HTML with the linked CSS file and Google Font
  html_with_css = f"""
  <!DOCTYPE html>
  <html>
  <head>
    <meta charset="UTF-8">
    <title>Resume</title>
    <link rel="stylesheet" href="{css_file}">
    <link rel="preconnect" href="https://fonts.googleapis.com">
    <link rel="preconnect" href="https://fonts.gstatic.com" crossorigin>
    <link href="https://fonts.googleapis.com/css2?family={google_font}&display=swap" rel="stylesheet">
  </head>
  <body>
    {html_output}
  </body>
  </html>
  """

  return html_with_css

# Example usage with resume_list and a Google Font:
html_output = markdown_to_html_with_css(resume_list[0], google_font="Open+Sans")

# Now you can save html_output to a file or use it further
print(html_output)

# Specify the filename
filename = "openai_resume.html"

# Write the HTML to the file
with open(filename, "w") as f:
  f.write(html_output)

print(f"HTML file saved as {filename}")


  <!DOCTYPE html>
  <html>
  <head>
    <meta charset="UTF-8">
    <title>Resume</title>
    <link rel="stylesheet" href="styles.css">
    <link rel="preconnect" href="https://fonts.googleapis.com">
    <link rel="preconnect" href="https://fonts.gstatic.com" crossorigin>
    <link href="https://fonts.googleapis.com/css2?family=Open+Sans&display=swap" rel="stylesheet">
  </head>
  <body>
    <p>```markdown</p>
<h1>Rod Morrison</h1>
<ul>
<li>Email: <a href="mailto:rod.morrison@gmail.com"><strong>rod.morrison@gmail.com</strong></a>  </li>
<li>Phone: <a href="tel:7139077102"><strong>713-907-7102</strong></a>  </li>
<li><a href="https://www.linkedin.com/in/rmorrison/"><strong>LinkedIn</strong></a>  </li>
<li><a href="https://github.com/Mojoryson"><strong>GitHub</strong></a>  </li>
</ul>
<hr />
<h2>Professional Summary</h2>
<p>Data-driven growth leader with extensive experience in AI, data analytics, and cloud platforms within cybersecurity contexts. Proven track record in transforming data

In [24]:
print(html_output)


  <!DOCTYPE html>
  <html>
  <head>
    <meta charset="UTF-8">
    <title>Resume</title>
    <link rel="stylesheet" href="styles.css">
    <link rel="preconnect" href="https://fonts.googleapis.com">
    <link rel="preconnect" href="https://fonts.gstatic.com" crossorigin>
    <link href="https://fonts.googleapis.com/css2?family=Open+Sans&display=swap" rel="stylesheet">
  </head>
  <body>
    <p>```markdown</p>
<h1>Rod Morrison</h1>
<ul>
<li>Email: <a href="mailto:rod.morrison@gmail.com"><strong>rod.morrison@gmail.com</strong></a>  </li>
<li>Phone: <a href="tel:7139077102"><strong>713-907-7102</strong></a>  </li>
<li><a href="https://www.linkedin.com/in/rmorrison/"><strong>LinkedIn</strong></a>  </li>
<li><a href="https://github.com/Mojoryson"><strong>GitHub</strong></a>  </li>
</ul>
<hr />
<h2>Professional Summary</h2>
<p>Data-driven growth leader with extensive experience in AI, data analytics, and cloud platforms within cybersecurity contexts. Proven track record in transforming data

In [25]:
# Save to PDF
output_resume_pdf = "resume/openai_resume_new.pdf"

# Convert HTML to PDF and save
HTML(string=html_output).write_pdf(output_resume_pdf, stylesheets=['styles.css'])

In [26]:
print(resume_list[0])

```markdown
# Rod Morrison

* Email: [**rod.morrison@gmail.com**](mailto:rod.morrison@gmail.com)  
* Phone: [**713-907-7102**](tel:7139077102)  
* [**LinkedIn**](https://www.linkedin.com/in/rmorrison/)  
* [**GitHub**](https://github.com/Mojoryson)  

---

## Professional Summary

Data-driven growth leader with extensive experience in AI, data analytics, and cloud platforms within cybersecurity contexts. Proven track record in transforming data ecosystems to optimize customer acquisition, retention, and product adoption. Expertise in driving the integration of predictive technologies, with an emphasis on enhancing security measures against cyber threats. Adept at leading cross-functional teams to tackle complex technical challenges and ensure seamless deployment of innovative solutions.

---

## Skills

- **Technical Expertise:** Python, AWS, GCP, SQL, ETL/ELT, API Integration, Data Analytics, Cloud Hosting Architectures, Cybersecurity Tools
- **Sales & Solutions Engineering:** Pre-sal

In [27]:
# save as markdown
output_file = "resume/openai_resume_new.md"

with open(output_file, "w", encoding="utf-8") as file:
    file.write(resume_list[0])

In [29]:
display(Markdown(resume_list[1]))



- **Skills to Strengthen Alignment**: 
  - Consider adding specific cybersecurity-related skills such as knowledge of DNS protocols and networking infrastructure to align with the job description.

- **Certifications or Courses**: 
  - Pursue certifications in cybersecurity (e.g., CompTIA Security+, Certified Information Systems Security Professional (CISSP)) to enhance credibility in the field.

- **Project Ideas or Experiences**: 
  - Engage in projects that focus on developing API integrations with cybersecurity tools, possibly through collaborations or hackathons, to demonstrate hands-on experience with relevant technologies.

---

**Evaluation**: **CLOSE**  
Rod Morrison meets most requirements for the Solution Engineer role at BforeAI, particularly in technical expertise, customer engagement, and solution integration. There are some minor gaps in cybersecurity-specific knowledge and practical experience with certain tools, but with targeted development, Rod could effectively transition into this role.
```